# Manipulação de Strings no Apache Spark

O Apache Spark fornece uma variedade de funções para manipular strings em DataFrames. Essas funções são úteis para realizar tarefas como limpeza, formatação, extração e transformação de dados de texto.

## Funções de Manipulação de Strings

As funções a seguir são comumente usadas para manipulação de strings:

| Função                       | Descrição                                                    | Exemplo de Caso de Uso                  |
|------------------------------|--------------------------------------------------------------|---------------------------------------|
| `concat(str1, str2, ...)`    | Concatena strings juntas                                     | Unir nomes em um único campo.          |
| `substring(str, pos, len)`   | Extrai uma parte da string a partir de uma posição e comprimento específicos | Obter prefixos, sufixos ou substrings. |
| `length(str)`                | Retorna o comprimento da string                              | Verificar a validade de identificadores. |
| `trim(str)`                  | Remove espaços em branco do início e do final da string     | Limpeza de dados de texto.             |
| `lower(str)`                 | Converte todos os caracteres para minúsculas                | Padronização de maiúsculas e minúsculas. |
| `upper(str)`                 | Converte todos os caracteres para maiúsculas                | Padronização de maiúsculas e minúsculas. |
| `initcap(str)`               | Converte a primeira letra de cada palavra para maiúscula    | Formatação de nomes.                   |
| `replace(str, target, replacement)` | Substitui uma sequência por outra                    | Correção de erros de digitação.         |
| `regexp_replace(str, pattern, replacement)` | Substitui padrões regulares por outra sequência | Extração e correção de informações.    |
| `split(str, pattern)`        | Divide a string com base em um padrão                       | Separar valores em uma lista.           |
| `instr(str, substr)`         | Encontra a posição da primeira ocorrência de uma subtring  | Validação de presença de informações.  |
| `substring_index(str, delimiter, count)` | Divide a string com base em um delimitador e recupera a subtring em uma posição específica | Separar e recuperar informações.  |
